In [1]:
import sys
sys.path.append('..')

In [2]:
import logging
import sys
from aequilibrae import Project
from aequilibrae.paths import TrafficAssignment, TrafficClass
from aequilibrae import logger

In [3]:
stdout_handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter("%(asctime)s ; %(levelname)s ; %(message)s")
stdout_handler.setFormatter(formatter)
logger.addHandler(stdout_handler)

In [4]:
pth = 'D:/release/Sample models/sioux_falls_multi-Class'

In [5]:
proj = Project()
proj.open(pth)

2021-02-26 22:16:33,992 ; INFO ; Opened project on D:/release/Sample models/sioux_falls_multi-Class


In [6]:
proj_matrices = proj.matrices
proj_matrices.update_database()
mat_list = proj_matrices.list()

In [7]:
mat_to_use = 'demand_mc_omx' #mat_list

In [8]:
proj.network.build_graphs()
carGraph = proj.network.graphs['c']
truckGraph = proj.network.graphs['T']
motoGraph = proj.network.graphs['M']

for graph in [motoGraph, carGraph, motoGraph]:
    graph.graph.loc[:, 'free_flow_time'] *= 60
    graph.graph.loc[graph.graph.free_flow_time==0, 'free_flow_time'] = 0.0001

2021-02-26 22:16:34,188 ; WARNING ; Field(s) name, lanes has(ve) at least one NaN value. Check your computations
2021-02-26 22:16:34,244 ; WARNING ; Field(s) name, lanes has(ve) at least one NaN value. Check your computations
2021-02-26 22:16:34,307 ; WARNING ; Field(s) name, lanes has(ve) at least one NaN value. Check your computations
2021-02-26 22:16:34,362 ; WARNING ; Field(s) name, lanes has(ve) at least one NaN value. Check your computations
2021-02-26 22:16:34,419 ; WARNING ; Field(s) name, lanes has(ve) at least one NaN value. Check your computations
2021-02-26 22:16:34,474 ; WARNING ; Field(s) name, lanes has(ve) at least one NaN value. Check your computations


In [9]:
carDemand = proj_matrices.get_matrix(mat_to_use)
carDemand.computational_view('car')
carClass = TrafficClass(carGraph, carDemand)
carClass.set_pce(1)
carClass.set_vot(1.0)
carClass.set_fixed_cost('toll', 0.025)

In [10]:
motoDemand = proj_matrices.get_matrix(mat_to_use)
motoDemand.computational_view('motorcycle')
motoClass = TrafficClass(motoGraph, motoDemand)
motoClass.set_pce(0.2)
motoClass.set_vot(1.0)
motoClass.set_fixed_cost('toll', 0.0125)

In [11]:
truckDemand = proj_matrices.get_matrix(mat_to_use)
truckDemand.computational_view('trucks')
truckClass = TrafficClass(truckGraph, truckDemand)
truckClass.set_pce(1.5)
truckClass.set_vot(2.0)
truckClass.set_fixed_cost('toll', 0.05)

In [12]:
assig = TrafficAssignment()
assig.set_classes([carClass, motoClass, truckClass])

assig.set_vdf("BPR")  # This is not case-sensitive # Then we set the volume delay function

assig.set_vdf_parameters({"alpha": "b", "beta": "power"})  # And its parameters

assig.set_time_field("free_flow_time")
assig.set_capacity_field(f"capacity")  # The capacity and free flow travel times as they exist in the graph

# And the algorithm we want to use to assign
assig.set_algorithm('bfw')

# since I haven't checked the parameters file, let's make sure convergence criteria is good
assig.max_iter = 400
assig.rgap_target = 0.0000001

assig.execute()  # we then execute the assignment

2021-02-26 22:16:35,329 ; INFO ; bfw Assignment STATS
2021-02-26 22:16:35,330 ; INFO ; Iteration, RelativeGap, stepsize


..\aequilibrae\paths\graph.py:445: UserWarning: Cost field with wrong type. Converting to float64
  warn("Cost field with wrong type. Converting to float64")


2021-02-26 22:16:35,658 ; INFO ; 1,inf,1.0
2021-02-26 22:16:35,986 ; INFO ; 2,0.9160240822531497,0.29510156145517613
2021-02-26 22:16:36,347 ; INFO ; 3,0.6857984396146746,2.4938084057565026e-17
2021-02-26 22:16:36,685 ; INFO ; 4,0.6452740427802999,0.16658162625432274
2021-02-26 22:16:37,024 ; INFO ; 5,0.39172900036301667,0.16477568865625894
2021-02-26 22:16:37,364 ; INFO ; 6,0.20730733607012222,0.2646035294101979
2021-02-26 22:16:37,708 ; INFO ; 7,0.22739190718369973,0.18585392437352066
2021-02-26 22:16:38,046 ; INFO ; 8,0.1648291956702236,0.26375173735547475
2021-02-26 22:16:38,389 ; INFO ; 9,0.22266705908873985,0.15611448576846962
2021-02-26 22:16:38,728 ; INFO ; 10,0.14773909241160235,0.03924090530658296
2021-02-26 22:16:39,065 ; INFO ; 11,0.11451476293903873,0.4458221938639476
2021-02-26 22:16:39,407 ; INFO ; 12,0.14519609807266506,0.10301950275082519
2021-02-26 22:16:39,749 ; INFO ; 13,0.08676550456574718,0.0033460604725207873
2021-02-26 22:16:40,090 ; INFO ; 14,0.0652255930837311

2021-02-26 22:17:12,417 ; INFO ; 108,0.00036260228541646505,0.002015291305769956
2021-02-26 22:17:12,761 ; INFO ; 109,0.000282025153626404,0.0006295299651811105
2021-02-26 22:17:13,101 ; INFO ; 110,0.0003093760074438779,0.005305167654155576
2021-02-26 22:17:13,442 ; INFO ; 111,0.0003220279267230049,0.005692820105083141
2021-02-26 22:17:13,793 ; INFO ; 112,0.00024003271053853477,0.006289606220902964
2021-02-26 22:17:14,130 ; INFO ; 113,0.00025281911958497646,0.004861881100064951
2021-02-26 22:17:14,473 ; INFO ; 114,0.00017546569954589433,0.01024808829028237
2021-02-26 22:17:14,814 ; INFO ; 115,0.0001968211755662053,0.006032862004566103
2021-02-26 22:17:15,154 ; INFO ; 116,0.00019624127861403751,0.006461728596173861
2021-02-26 22:17:15,498 ; INFO ; 117,0.00010887774110209487,0.00543761882954672
2021-02-26 22:17:15,838 ; INFO ; 118,0.00012285598288793313,0.005690415726864603
2021-02-26 22:17:16,180 ; INFO ; 119,0.00011855792020740683,0.009012502736021855
2021-02-26 22:17:16,523 ; INFO ; 1

2021-02-26 22:17:34,314 ; WARNING ; # Alert: Adding 0.01% of 5.813953488372093e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:17:34,315 ; INFO ; 172,4.290900758702354e-06,5.813953488372093e-07
2021-02-26 22:17:34,656 ; WARNING ; # Alert: Adding 0.01% of 5.780346820809248e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:17:34,658 ; INFO ; 173,4.244594795407733e-06,5.780346820809248e-07
2021-02-26 22:17:35,006 ; INFO ; 174,4.188742568032798e-06,3.7892375910158343e-05
2021-02-26 22:17:35,346 ; INFO ; 175,6.480081100709671e-06,1.3646960215840015e-05
2021-02-26 22:17:35,689 ; INFO ; 176,5.208279719427305e-06,6.0529952397398165e-05
2021-02-26 22:17:36,047 ; INFO ; 177,1.355993024297444e-05,8.039982291014863e-05
2021-02-26 22:17:36,385 ; WARNING ; # Alert: Adding 0.01% of 5.617977528089887e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:17:36,387 ;

...

2021-02-26 22:17:58,638 ; INFO ; 243,7.132949571629182e-06,4.11522633744856e-07
2021-02-26 22:17:58,980 ; WARNING ; # Alert: Adding 0.01% of 4.0983606557377047e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:17:58,982 ; INFO ; 244,7.187534205466717e-06,4.0983606557377047e-07
2021-02-26 22:17:59,321 ; WARNING ; # Alert: Adding 0.01% of 4.0816326530612243e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:17:59,322 ; INFO ; 245,7.252319819462328e-06,4.0816326530612243e-07
2021-02-26 22:17:59,665 ; WARNING ; # Alert: Adding 0.01% of 4.065040650406504e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:17:59,667 ; INFO ; 246,7.3065769664596265e-06,4.065040650406504e-07
2021-02-26 22:18:00,007 ; WARNING ; # Alert: Adding 0.01% of 4.048582995951417e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:18:00,010 

2021-02-26 22:18:10,272 ; INFO ; 277,9.055252376883601e-06,3.6101083032490974e-07
2021-02-26 22:18:10,609 ; WARNING ; # Alert: Adding 0.01% of 3.5971223021582736e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:18:10,610 ; INFO ; 278,9.097650071308872e-06,3.5971223021582736e-07
2021-02-26 22:18:10,951 ; WARNING ; # Alert: Adding 0.01% of 3.5842293906810036e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:18:10,953 ; INFO ; 279,9.146678769932923e-06,3.5842293906810036e-07
2021-02-26 22:18:11,292 ; WARNING ; # Alert: Adding 0.01% of 3.5714285714285716e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:18:11,293 ; INFO ; 280,9.19495152285435e-06,3.5714285714285716e-07
2021-02-26 22:18:11,633 ; WARNING ; # Alert: Adding 0.01% of 3.558718861209964e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:18:11,63

2021-02-26 22:18:21,898 ; INFO ; 311,1.0484842222285789e-05,3.2154340836012864e-07
2021-02-26 22:18:22,255 ; WARNING ; # Alert: Adding 0.01% of 3.205128205128205e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:18:22,257 ; INFO ; 312,1.0521671663960749e-05,3.205128205128205e-07
2021-02-26 22:18:22,593 ; WARNING ; # Alert: Adding 0.01% of 3.194888178913738e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:18:22,595 ; INFO ; 313,1.0558307107582582e-05,3.194888178913738e-07
2021-02-26 22:18:22,935 ; WARNING ; # Alert: Adding 0.01% of 3.184713375796178e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:18:22,937 ; INFO ; 314,1.0591056730781508e-05,3.184713375796178e-07
2021-02-26 22:18:23,278 ; WARNING ; # Alert: Adding 0.01% of 3.174603174603175e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:18:23,280

2021-02-26 22:18:33,532 ; INFO ; 345,1.1591264296332795e-05,2.898550724637681e-07
2021-02-26 22:18:33,873 ; WARNING ; # Alert: Adding 0.01% of 2.890173410404624e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:18:33,875 ; INFO ; 346,1.1620881455502452e-05,2.890173410404624e-07
2021-02-26 22:18:34,215 ; WARNING ; # Alert: Adding 0.01% of 2.881844380403458e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:18:34,217 ; INFO ; 347,1.1651483522362603e-05,2.881844380403458e-07
2021-02-26 22:18:34,558 ; WARNING ; # Alert: Adding 0.01% of 2.8735632183908047e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:18:34,559 ; INFO ; 348,1.1678232740319571e-05,2.8735632183908047e-07
2021-02-26 22:18:34,899 ; WARNING ; # Alert: Adding 0.01% of 2.865329512893983e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:18:34,90

2021-02-26 22:18:45,152 ; INFO ; 379,1.2334141755811946e-05,2.6385224274406334e-07
2021-02-26 22:18:45,492 ; WARNING ; # Alert: Adding 0.01% of 2.631578947368421e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:18:45,494 ; INFO ; 380,1.2352637795841915e-05,2.631578947368421e-07
2021-02-26 22:18:45,834 ; WARNING ; # Alert: Adding 0.01% of 2.624671916010499e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:18:45,835 ; INFO ; 381,1.2372170972746104e-05,2.624671916010499e-07
2021-02-26 22:18:46,176 ; WARNING ; # Alert: Adding 0.01% of 2.617801047120419e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:18:46,177 ; INFO ; 382,1.239246057222964e-05,2.617801047120419e-07
2021-02-26 22:18:46,519 ; WARNING ; # Alert: Adding 0.01% of 2.610966057441253e-07 as step size to make it non-zero. ('f(a) and f(b) must have different signs',)
2021-02-26 22:18:46,521 

In [15]:
df = assig.report()
df

,iteration,rgap,alpha,warnings,beta0,beta1,beta2
0,1,inf,1.000000e+00,,1.0,0.0,0.0
1,2,0.916024,2.951016e-01,,1.0,0.0,0.0
2,3,0.685798,2.493808e-17,,1.0,0.0,0.0
3,4,0.645274,1.665816e-01,,1.0,0.0,0.0
4,5,0.391729,1.647757e-01,,1.0,0.0,0.0
...,...,...,...,...,...,...,...
395,396,0.000007,2.525253e-07,,-1.0,-1.0,-1.0
396,397,0.000007,2.518892e-07,,-1.0,-1.0,-1.0
397,398,0.000008,2.512563e-07,,-1.0,-1.0,-1.0
398,399,0.000013,2.506266e-07,,-1.0,-1.0,-1.0


In [14]:
proj.close()

2021-02-26 22:18:52,559 ; INFO ; Closed project on D:/release/Sample models/sioux_falls_multi-Class
